In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KernelDensity
import folium
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
import sys
!{sys.executable} -m pip install scikit-learn folium

  Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached folium-0.20.0-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached scipy-1.16.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached branca-0.8.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached xyzservices-2025.11.0-py3-none-any.whl.metadata (4.3 kB)
Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl (8.1 MB)
Using cached folium-0.20.0-py2.py3-none-any.whl (113 kB)
Using cached branca-0.8.2-py3-none-any.whl (26 kB)
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
Using cached scipy-1.16.3-cp311-cp311-win_amd64.whl (38.7 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Using cached xyzservices-2025.11.0-py3-none-any.whl (93 kB)

   ----- ---------------------------------- 1/7 [threadpoolctl]
   ----------- ---------------------------- 2

In [4]:
import os

In [6]:


# اصلاح نام فایل به cleaned_data.csv (نامی که preprocess.py تولید می‌کند)
INPUT_FILE = '../data/cleaned_data.csv'
OUTPUT_FILE = '../data/analyzed_data.csv'

if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE)
    print(f"✅ Data loaded successfully. Shape: {df.shape}")
else:
    print(f"❌ Error: File not found at {INPUT_FILE}")
    print("Please run 'preprocess.py' first.")

✅ Data loaded successfully. Shape: (12818, 53)


In [7]:
print("Running DBSCAN clustering...")

# گرفتن مختصات (توجه: ترتیب Longitude و Latitude برای برخی الگوریتم‌ها مهم است)
coords = df[['Longitude', 'Latitude']].values

# تنظیمات DBSCAN
# eps=0.01: شعاع همسایگی (حدود ۱ کیلومتر)
# min_samples=10: حداقل تعداد تصادف برای تشکیل یک کانون خطر
db = DBSCAN(eps=0.01, min_samples=10).fit(coords)

# افزودن ستون Cluster به دیتافریم
df['Cluster'] = db.labels_

# گزارش نتایج
n_clusters = len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)
n_noise = list(db.labels_).count(-1)

print(f"📍 Total Clusters found: {n_clusters}")
print(f"🗑️ Noise points (outliers): {n_noise}")

Running DBSCAN clustering...
📍 Total Clusters found: 111
🗑️ Noise points (outliers): 10182


In [8]:
print("Running KDE for density estimation...")

# اجرای KDE با کرنل گوسی
kde = KernelDensity(kernel='gaussian', bandwidth=0.01).fit(coords)

# محاسبه امتیاز لگاریتمی
log_dens = kde.score_samples(coords)

# تبدیل به چگالی نرمال و ذخیره در ستون Density
df['Density'] = np.exp(log_dens)

print("✅ KDE calculation complete. 'Density' column added.")
print(df[['Latitude', 'Longitude', 'Cluster', 'Density']].head())

Running KDE for density estimation...
✅ KDE calculation complete. 'Density' column added.
    Latitude  Longitude  Cluster   Density
0  53.488762  -1.117674       -1  0.388904
1  53.488762  -1.117674       -1  0.388904
2  51.508536   0.063809       -1  1.367807
3  51.547797  -0.393649      110  1.125135
4  51.492533  -0.416336       -1  1.621488


In [9]:
df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Analyzed data saved to: {OUTPUT_FILE}")

✅ Analyzed data saved to: ../data/analyzed_data.csv


In [10]:
# ساخت نقشه با تمرکز بر میانگین مختصات داده‌ها
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=6)

# فیلتر کردن نقاط نویز (فقط نمایش نقاط خطرناک)
high_risk_points = df[df['Cluster'] != -1]

print(f"🗺️ Plotting {len(high_risk_points)} high-risk points...")

for _, row in high_risk_points.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3,
        color='red',
        fill=True,
        fill_color='darkred',
        fill_opacity=0.7,
        popup=f"Cluster: {row['Cluster']} | Density: {row['Density']:.4f}"
    ).add_to(m)

# نمایش نقشه
m

🗺️ Plotting 2636 high-risk points...
